In [27]:
from zeep import Client
import zeep
from zeep.wsse.username import UsernameToken

In [ ]:
client = Client('https://servicoshm.saude.gov.br/cnes/EstabelecimentoSaudeService/v1r0?wsdl', wsse=UsernameToken('CNES.PUBLICO', 'cnes#2015public'))

In [ ]:
##Mostrar as informações do WSDL
client.wsdl.dump()

In [ ]:
request_data = {
    'FiltroPesquisaEstabelecimentoSaude': {
        'CodigoCNES': {
            'codigo': 2653982
        }
    },
}

In [ ]:
response = client.service.consultarEstabelecimentoSaude(**request_data)

In [ ]:
print (response)

In [11]:
import pandas

In [ ]:
data_frame = pandas.read_csv('listas/municipios.csv', sep=';')

In [ ]:
data_frame

In [ ]:
data_frame.MUNICIPIO

In [9]:
request_data_municipio = {
    'Municipio': {
        'codigoMunicipio' : 240810
    }
}

In [8]:
client_cnes_service = Client('https://servicos.saude.gov.br/cnes/CnesService/v1r0?wsdl', wsse=UsernameToken('CNES.PUBLICO', 'cnes#2015public'))

In [ ]:
client_cnes_service.wsdl.dump()

In [10]:
response = client_cnes_service.service.consultarEstabelecimentoSaudePorMunicipio(**request_data_municipio)

In [29]:
blau = zeep.helpers.serialize_object(response)

In [30]:
blau

[OrderedDict([('DadosBasicosEstabelecimento',
               [OrderedDict([('CodigoCNES',
                              OrderedDict([('codigo', '7700490')])),
                             ('NomeFantasia',
                              OrderedDict([('Nome',
                                            'CONSIULTORIO MEDICO DRA ANA TERESA')]))]),
                OrderedDict([('CodigoCNES',
                              OrderedDict([('codigo', '7698127')])),
                             ('NomeFantasia',
                              OrderedDict([('Nome',
                                            'CONSULTORIO DE PSICOLOGIA DRA CLAUDIA MONTENEGRO')]))]),
                OrderedDict([('CodigoCNES',
                              OrderedDict([('codigo', '7707150')])),
                             ('NomeFantasia',
                              OrderedDict([('Nome',
                                            'INSTITUTO DO SONO E TRANSTORNOS MENTAIS')]))]),
                OrderedDict([('CodigoC

In [ ]:
for row in data_frame.itertuples():
    path = 'resultados/%s/%s' % (row.UF, row.MUNICIPIO) 
    %mkdir $path